## Imports

In [41]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium

## Read-in

In [42]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

df = pd.read_csv('South Florida Condo-Hotels - Sheet1.csv')

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

## Geocode

In [43]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [44]:
def geocode(add):
    g = gmaps_key.geocode(add)
    lat = g[0]["geometry"]["location"]["lat"]
    lng = g[0]["geometry"]["location"]["lng"]
    return (lat, lng)

df['geocoded'] = df['Location_Address'].apply(geocode)

In [45]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].str.strip('()').str.split(', ', expand=True)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

## Format Data

In [46]:
### Insert NaNs if needed ###
df = df.replace('N/A', np.nan)

In [47]:
def popup_html(row):
    property_name = row['Property Name']
    address = row['Location_Address']
    description = row['Description']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Name: </strong>{}'''.format(property_name) + '''<br>
    <strong>Address: </strong>{}'''.format(address) + '''<br>
    <strong>Description: </strong>{}'''.format(description) + '''<br>
    </html>
    '''
    return html

## Correction Section

In [56]:
df.at[7,'lat']=(25.940847639678452)
df.at[7,'lon']=(-80.12100834628404)

In [57]:
import folium
import branca

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=10)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'South Florida Condo-Hotels')

for index, row in df.iterrows():
    html = popup_html(row)
    popup = folium.Popup(html, max_width=400)
    folium.Marker([row['lat'], row['lon']], popup=popup).add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.fit_bounds(bounds=df[['lat', 'lon']].mean().to_list(), max_zoom=15)
m

In [52]:
m.save('index.html')

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [53]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [54]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/SouthFloridaCondoHotels
